### Reading images

In [7]:
def read_image(source_path):
    #print(source_path)
    filename = source_path.split('/')[-1]
    #print(filename)
    filename = './data/IMG/'+ filename.split('\\')[-1]
    #print(filename)
    image = cv2.imread(filename)
    return image

In [15]:
import csv
import cv2
import numpy as np
from tqdm import tqdm

lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
#for line in lines:
for i in tqdm(range(0, len(lines), 1)):
    steering_center = float(lines[i][3])
    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # Center
    images.append(read_image(lines[i][0]))
    measurements.append(steering_center)
    # Left
    images.append(read_image(lines[i][1]))
    measurements.append(steering_left)
    # Right
    images.append(read_image(lines[i][2]))
    measurements.append(steering_right)

100%|██████████| 8036/8036 [00:21<00:00, 374.32it/s]


### Image augmentation

In [17]:
augmented_images, augmented_measurements = [], []
# for image, measurement in zip(images, measurements, len(images)):
for i in tqdm(range(0, len(images), 1)):
    image = images[i]
    measurement = measurements[i]
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, flipCode=1))
    augmented_measurements.append(measurement*-1.0)

100%|██████████| 24108/24108 [00:04<00:00, 5116.80it/s]


### Model

In [20]:
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Cropping2D

model = Sequential()
# Normalize and mean centering the data 
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Conv2D(24, (5, 5), strides=(2,2), activation = 'relu'))
model.add(Conv2D(36, (5, 5), strides=(2,2), activation = 'relu'))
model.add(Conv2D(48, (5, 5), strides=(2,2), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3, verbose = 2)
model.save('model.h5')

Train on 38572 samples, validate on 9644 samples
Epoch 1/3
195s - loss: 0.0178 - val_loss: 0.0175
Epoch 2/3
193s - loss: 0.0146 - val_loss: 0.0205
Epoch 3/3
191s - loss: 0.0136 - val_loss: 0.0194


### History

In [21]:
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

NameError: name 'train_generator' is not defined